# Movie Review sentiment analysis on the IMDB dataset

In [9]:
import matplotlib.pyplot as plt
import os
import nltk
%matplotlib inline
plt.rcParams['figure.figsize'] = (5, 4)

In [10]:
# download the IMDB large movie review corpus from the class webpage to a file location on your computer

PATH_TO_DATA = 'large_movie_review_dataset'  # set this variable to point to the location of the IMDB corpus on your computer
POS_LABEL = 'pos'
NEG_LABEL = 'neg'
TRAIN_DIR = os.path.join(PATH_TO_DATA, "train")
TEST_DIR = os.path.join(PATH_TO_DATA, "test")

for label in [POS_LABEL, NEG_LABEL]:
    if len(os.listdir(TRAIN_DIR + "/" + label)) == 12500:
        print "Great! You have 12500 {} reviews in {}".format(label, TRAIN_DIR + "/" + label)
    else:
        print "Oh no! Something is wrong. Check your code which loads the reviews"

Great! You have 12500 pos reviews in large_movie_review_dataset\train/pos
Great! You have 12500 neg reviews in large_movie_review_dataset\train/neg


In [11]:
# Actually reading the data you are working with is an important part of NLP! Let's look at one of these reviews

print open(TRAIN_DIR + "/neg/3740_2.txt").read()

Right away, this film was ridiculous. Not that it didn't have redeeming aspects For example, the best thing about this film was the beautiful background scenery. Anyone not living on the East Coast should know the South doesn't have beautiful mountains like those found in the West. I knew it was Utah right off the bat, but perhaps Dalton couldn't suppress his English accent, so they had to excuse it by saying this was a southern town. Subverting his accent into a Southern one was easier. Sure the film has plot twists, but its phony sense of place was something I couldn't get past. It's not like Utah doesn't have meth labs... so why the writers thought it necessary to pretend it was in the South is beyond me. <br /><br />One other thing in action pictures always puzzles me. Why do they always make the "cocking" sound effect when the character pulls out an automatic handgun? It seemed every other sound effect in this movie was a "chuk-chich" signifying a 9mm was loaded and ready to fire

In [12]:
from nltk.corpus import stopwords
import re
from collections import Counter
def tokenize_doc_and_more(doc):
    bow = defaultdict(float)
    
    # Converting into lower case text
    doc_lower = doc.lower() 
    
    # removing pucntuations ("..", ".", ",")
    doc_wo_punc = re.sub(r'(\.+$|\?+$|\,|\'|\.{2,}|<br *(/>)?)',"",doc_lower)
    
    # removing two spaces 
    text_with_one_space = re.sub(r'[ ]{2,}'," ", doc_wo_punc).split()
    
    # removing stop words like "and", "the"
    stop_words = set(stopwords.words('english'))
    filtered_sentence = [w for w in text_with_one_space if not w in stop_words]
    
    bow = Counter(filtered_sentence)
    return dict(bow)

In [13]:
import glob
from collections import defaultdict
word_counts = defaultdict(float)  # you might want to use a defaultdict instead https://docs.python.org/2/library/collections.html
                  # defaultdicts are often useful for NLP in python
document_new = []
for label in [POS_LABEL, NEG_LABEL]:
    for directory in [TRAIN_DIR, TEST_DIR]:
        for fn in glob.glob(directory + "/" + label + "/*txt"):
            ## Implement me!
            temp = tokenize_doc_and_more(open(fn).read())
            document_new.append((list(temp), label))
            for key, item in temp.items():
                word_counts[key] += item

In [14]:
word_features = list(word_counts)[:4000]

In [15]:
def document_features(document): 
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [16]:
featuresets = [(document_features(d), c) for (d,c) in document_new]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [17]:
print(nltk.classify.accuracy(classifier, test_set))

0.75


In [30]:
classifier.show_most_informative_features(20)

Most Informative Features
        contains(devgan) = True              neg : pos    =     13.6 : 1.0
    contains(atrocious.) = True              neg : pos    =     11.9 : 1.0
  contains(wonderfully.) = True              pos : neg    =      9.4 : 1.0
       contains(quibble) = True              pos : neg    =      9.3 : 1.0
      contains(flawless) = True              pos : neg    =      8.4 : 1.0
      contains(fatalism) = True              pos : neg    =      8.4 : 1.0
        contains(quarry) = True              neg : pos    =      7.6 : 1.0
       contains(allegra) = True              pos : neg    =      7.0 : 1.0
     contains(overjoyed) = True              pos : neg    =      7.0 : 1.0
     contains(spongebob) = True              neg : pos    =      7.0 : 1.0
      contains(captures) = True              pos : neg    =      6.6 : 1.0
         contains((dawn) = True              pos : neg    =      6.4 : 1.0
      contains(descent") = True              neg : pos    =      6.3 : 1.0

In [47]:
def classify(review):
    return classifier.classify(document_features(tokenize_doc_and_more(review)))

In [48]:
# Positive review example
review = "I was looking forward to The Guardian, but when I walked into the theater I wasn't really in the mood for it at that particular time. It's kind of like the Olive Garden - I like it, but I have to be in the right mindset to thoroughly enjoy it.<br /><br />I'm not exactly sure what was dampening my spirit. The trailers looked good, but the water theme was giving me bad flashbacks to the last Kevin Costner movie that dealt with the subject - Waterworld. Plus, despite the promise Ashton Kutcher showed in The Butterfly Effect, I'm still not completely sold on him. Something about the guy just annoys me. Probably has to do with his simian features.<br /><br />It took approximately two minutes for my fears to subside and for my hesitancies to slip away. The movie immediately throws us into the midst of a tense rescue mission, and I was gripped tighter than Kenny Rogers' orange face lift. My concerns briefly bristled at Kutcher's initial appearance due to the fact that too much effort was made to paint him as ridiculously cool and rebellious. Sunglasses, a tough guy toothpick in his mouth, and sportin' a smirk that'd make George Clooney proud? Yeah, we get it. I was totally ready to hate him.<br /><br />But then he had to go and deliver a fairly strong performance and force me to soften my jabs. <br /><br />Darn you, ape man! Efficiently mixing tense, exciting rescue scenes, drama, humor, and solid acting, The Guardian is easily a film that I dare say the majority of audiences will enjoy. You can quibble about its clichés, predictability, and rare moments of overcooked sappiness, but none of that takes away from the entertainment value.<br /><br />I had a bad feeling that the pace would slow too much when Costner started training the young guys, but on the contrary, the training sessions just might be the most interesting aspect of the film. Coast Guard Rescue Swimmers are heroes whose stories have never really been portrayed on the big screen, so I feel the inside look at what they go through and how tough it is to make it is very informative and a great way to introduce audiences to this under-appreciated group.<br /><br />Do you have what it takes to be a rescue swimmer? Just think about it -you get to go on dangerous missions in cold, dark, rough water, and then you must fight disorientation, exhaustion, hypothermia, and a lack of oxygen all while trying to help stranded, panicked people who are depending on you for their survival. And if all that isn't bad enough, sometimes you can't save everybody so you have to make the tough decision of who lives and who dies.<br /><br />Man, who wants all that responsibility? Not me! I had no idea what it was really like for these guys, and who would have thought I'd have an Ashton Kutcher/Kevin Costner movie to thank for the education? <br /><br />Not only does The Guardian do a great job of paying tribute to this rare breed of hero, but lucky for us it also does a good job of entertaining its paying customers.<br /><br />THE GIST <br /><br />Moviegoers wanting an inside look at what it's like to embark on a daring rescue mission in the middle of the ocean might want to give The Guardian a chance. I saw it for free, but had I paid I would've felt I had gotten my money's worth."

In [49]:
classify(review)

'pos'

In [56]:
review = "An obvious vanity press for Julie in her first movie with Blake. Let's see. Where do we begin. She is a traitor during a world war; she redeems that by falling in love; her friends (who are presumably patriots because they are German citizens) are expendable and must die; and she winds up as a heroine. OK. The scenes with the drunken pilot and the buffoons who work for French intelligence can't even be described, and we won't even mention Rock's romantic scenes with a female. (By the way, when they visit a museum, look at his gaze - I reran it on video and it's priceless). Is it a farce or is it a romantic classic or is it a war movie? I don't know and you won't either."

In [57]:
classify(review)

'neg'

# review downloaded from the internet for the movie marsall, this person gave 7/10 to the movie lets see what the classifier predicts

In [59]:
# review downloaded from the internet for the movie marsall, this person gave 7/10 to the movie lets see what the classifier predicts
review = "I got to attend an early screening of Marshall tonight. I'm interested to see how critics react. I have a feeling many of them will object to the paint-by-numbers approach to the film. While we have not seen Thurgood Marshall represented much in film, it does feel like we've seen this movie more than once before. But that isn't really the point. I've eaten spaghetti and meatballs hundreds of times before. I still enjoy it each time, the same dish, so long as it is made well. And Marshall, while not reinventing any wheels, is made well. Chadwick Boseman leads a terrific cast that includes Josh Gad, Dan Stevens, James Cromwell, Kate Hudson and Sterling K. Brown. Everyone is there to give this very important true story some depth and weight. At the same time, the screenplay never gets too caught up in its own self-importance. While some very dark themes and tragic events are present, there is a sense of humor pervading much of the film. This makes the people and events portrayed in Marshall relatable, instead of feeling like we're watching a group of untouchable, stoic historical figures. Marshall isn't designed to inspire anger or guilt, instead it encourages us to examine examples of unity that have been used to overcome struggle. It has more in common with films like The Help or Hidden Figures, than more aggressive films like Detroit (though that film is very intense and impressive). I would say Marshall will play out just as well at home as it does in a theater, but there is something about seeing it with a crowd that in this case adds to the experience. The gasps of the audience when an atrocity is displayed, the clapping when a bigot loses his/her battle-it is a good film to enjoy with an audience. From a technical standpoint, the film does not go out of its way to impress. The cinematography, costume and production design, music, editing-all seems serviceable if not particularly memorable. In this case its the story and the figures it portrays that you'll remember."

In [60]:
classify(review)

'pos'

# The classifier successfully predicts the positive sentiment in the new review

In [68]:
review = "When it comes to remaking a movie, I'm all for it if it means that they're going to try and make a better movie out of something that wasn't all that impressive to begin with. That being said, if the original film was already solid or decently received by both audiences and critics, then why bother? Flatliners was a film that was released back in 1990, and I quite enjoy that film, even though the overall product has many issues of its own. I didn't see the reason for a remake, but I could see potential in improving it, so I was open-minded. Sadly, Flatliners is one of the worst films I've seen all year. Taking a solid premise and putting a supernatural spin on it for absolutely no apparent reason, bothered me to no end. Here is why Flatliners fails as both a remake and as an original piece to be shown to a new audience. The idea of doctors being capable of flatlining people and bringing them back to life, being able to have conversations about what death is like and going through hallucinations as a side effect is quite interesting; However, this version of the film becomes a supernatural thriller by the time it reaches its third act, making for a very confusing film, due to the fact that there is clearly no physical entity that could ever accomplish these things. This version of this concept just strips away anything that was exciting or intriguing about the original film. Not to compare and contrast, but idea of Flatliners definitely benefits from a more subdued and subtle approach to things. What bothered me was the fact that the majority of the cast seemed capable of being subdued, but the film's screenplay was such a mess that I found myself thinking these actors/actresses deserved better material. In particular, Diego Luna and Ellen Page were actually very good in their respective roles, making for a few emotionally resonant moments, even though the lines they were given were pretty lame. Quite honestly, with a better script, a title change, and a bit of originality, this cast could've worked in a much better movie. Even though the performances are all decent, the fact that this cast was a bunch of youngsters actually annoyed me. The original film was about a group of experienced doctors who had a neat idea, and were much more capable of being able to bring each other back to life. This time around, it's a group of students who have just enough knowledge in maybe being able to bring each other back. This notion alone was a scripting mistake, because it just becomes a story about naive young students who become obsessed with someone's experiment. I found no attachment to any of these characters and none of them really had a reason for wanting to die (with the exception of one or two without spoiling anything), which left me not caring from frame one. In the end, this film benefits from a strong enough cast (for the most part) and the concept itself is very interesting, but all you have to do is watch the original to see how it should be done. This film tries too many new things, and quite frankly fails at pretty much all of them. Having terrible dialogue, an unnecessary supernatural turn of events, and a climax that turns into a straight up horror flick, I found myself not caring what the outcome for each of the characters would be. The only thing redeemable about this film is the premise itself, which has been done better in the past, so I can't recommend this movie to anyone, but I do recommend checking out the original Flatliners if you haven't seen it yet."

# And the classifier fails to predict this review as a negative review.

In [69]:
classify(review)

'pos'